# ArekuNoimar/Llama-3.2-1B-Instruct-QA-v1-LoRA

### This is a program that performs LoRA parameter tuning.
### No Quantization

In [1]:
import time
program_start_time = time.time()

In [ ]:
password = input("Enter the password: ")
!echo {password} | sudo -S apt-get install mpich -y

In [ ]:
!pip install --upgrade pip  # Upgrade pip to the latest version
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124  # PyTorch libraries for deep learning, including vision and audio processing, optimized for CUDA 12.4
!pip install 'transformers==4.51.3'  # Library for state-of-the-art natural language processing models, including BERT, GPT, and others
!pip install 'accelerate==1.6.0'  # Library to simplify distributed training and inference of machine learning models
!pip install 'peft==0.15.2'  # Parameter-efficient fine-tuning library for transformer models
!pip install 'wandb==0.19.10'  # For machine learning experiment tracking and visualization
!pip install 'GPUtil==1.4.0'  # For GPU monitoring and manual logging of GPU utilization
!pip install 'psutil==7.0.0'  # For monitoring system resources and manual logging of CPU and memory usage
!pip install 'datasets==3.5.0'  # Library for easily accessing and managing datasets for machine learning
!pip install 'numpy==2.1.2'  # Fundamental library for numerical computations in Python, supports arrays and mathematical operations
!pip install 'ipywidgets==8.1.6'  # Library for creating interactive widgets in Jupyter Notebook
!pip install 'pandas==2.2.3'  # Library for data analysis and manipulation, provides DataFrame for tabular data
!pip install 'matplotlib==3.10.1'  # Library for data visualization, can draw various graphs such as line and scatter plots
!pip install 'deepspeed==0.16.7' --use-pep517  # Deep learning optimization library for distributed training, supports large-scale models and efficient resource utilizationa
!pip install 'pynvml==11.5.0'
!pip install 'ipykernel==6.29.4'
!pip install 'mpi4py'

In [4]:
# Set Environment Variables
default_environment_variables = {
    "code_file": "llama3-2-1b-instruct-qa-v1-lora.ipynb",
    "huggingface_token": "",
    "huggingface_repo_name": "llama-3-2-1b-instruct-qa-v1-lora",
    "huggingface_account_name": "ArekuNoimar",
    "wandb_account_name": "arekunoimar-deepspeed",
    "wandb_project_name": "llama-3-2-1b",
    "model_name": "meta-llama/Llama-3.2-1B",
    "dataset": "../dataset/alpaca_data.json",
    "dataset_max_length": 512,
    "apply_dataset_rate": 1.0,
    "dataset_train_rate": 0.8,
    "dataset_validation_rate": 0.1,
    "num_train_epochs": 3,
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "gradient_accumulation_steps": 1,
    "optim": "adamw_torch",
    "logging_steps": 1,
    "learning_rate": 2e-5,
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.06,
    "seed": 1024,
    "fp16": False,
    "bf16": True,
    "save_strategy": "steps",
    "save_steps": 10000,
    "save_total_limit": 1,
    "evaluation_strategy": "steps",
    "eval_steps": 10000,
    "do_eval": True,
    "logging_distance_time": 1,
    "deepspeed_zero0": False,
    "deepspeed_train_config_zero0_path":"deepspeed_train_config_zero0.json",
    "deepspeed_zero1": False,
    "deepspeed_train_config_zero1_path":"deepspeed_train_config_zero1.json",
    "deepspeed_zero2": True,
    "deepspeed_train_config_zero2_path":"deepspeed_train_config_zero2.json",
    "deepspeed_zero3": False,
    "deepspeed_train_config_zero3_path":"deepspeed_train_config_zero3.json",
    "deepspeed_zero3_infinity": False,
    "deepspeed_train_config_zero3_infinity_path":"deepspeed_train_config_zero3_infinity.json",
    "lora_r": 512,
    "lora_alpha": 1024,
    "lora_dropout": 0,
    "lora_target_modules": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "down_proj", "up_proj"]
}

# set constant
NO_INPUT_PATTERNS = {"<no input>", "<No input>", "<No Input>","<noinput>", "<Noinput>", "<NOINPUT>","<no input>.", "<noinput>.", "<noinput>,","[No Input]", "(no input)", "(No Input)","\"<noinput>\"", "\"<no input>\"", "`<noinput>`","no input", "No input", "No Input","No input.", "No Input.","no input required", "No input required", "No input required."}

In [ ]:
import pandas
df = pandas.read_json(default_environment_variables["dataset"], encoding="UTF-8")
df.head()

In [6]:
# import torch
# torch.backends.cuda.matmul.allow_tf32 = True # allow matrix multiplication in TF32
# torch.backends.cudnn.allow_tf32 = True # allow convolution in TF32

In [ ]:
import json
from transformers import LlamaForCausalLM, AutoTokenizer, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig
from collections import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt

class TrainUtil:

  def __init__(self, environment_valiables):
      self.alpaca_dataset = environment_valiables["dataset"]
      self.dataset_max_length = environment_valiables["dataset_max_length"]
      self.apply_dataset_rate = environment_valiables["apply_dataset_rate"]
      self.dataset_train_rate = environment_valiables["dataset_train_rate"]
      self.dataset_validation_rate = environment_valiables["dataset_validation_rate"]
      self.lora_r = environment_valiables["lora_r"]
      self.lora_alpha = environment_valiables["lora_alpha"]
      self.lora_dropout = environment_valiables["lora_dropout"]
      self.lora_target_modules = environment_valiables["lora_target_modules"]
      
  def read_alpaca_json_dataset(self) -> list:
      with open(self.alpaca_dataset, 'r', encoding='utf-8') as f:
          dataset = json.load(f)
      return dataset
  
  def clean_alpaca_dataset(self, dataframe):
      cleaned_df = dataframe.copy()
      for column in ['instruction', 'input', 'output']:
          if column in cleaned_df.columns:
              cleaned_df[column] = cleaned_df[column].astype(str).str.replace('\n', ' ') # replace \n to space
              cleaned_df[column] = cleaned_df[column].str.replace('\r', ' ') # replace \r to space
              cleaned_df[column] = cleaned_df[column].str.replace(r'\s+', ' ', regex=True) # replace continuous spaces to single space
              cleaned_df[column] = cleaned_df[column].str.strip() # remove leading and trailing spaces
      return cleaned_df
  
  def normalize_input_column_alpaca_dataset(self, dataframe):
      normalized_df = dataframe.copy()
      for normalize_text in NO_INPUT_PATTERNS:
          normalized_df['input'] = normalized_df['input'].str.replace(normalize_text, "", regex=True) # replace continuous spaces to single space
      normalized_df['input'] = normalized_df['input'].str.strip() # remove leading and trailing spaces
      return normalized_df

  def format_alpaca_dataset(self, dataframe):
      formatted_results = []
      for i in range(len(dataframe)):
          example_dict = dataframe.iloc[i].to_dict()
          instruction = example_dict["instruction"]
          user_input = example_dict["input"]
          output = example_dict["output"]

          if user_input:
              formatted_text = (
                  f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>"
                  f"{instruction}<|eot_id|>"
                  f"<|start_header_id|>user<|end_header_id|>"
                  f"{user_input}<|eot_id|>"
                  f"<|start_header_id|>assistant<|end_header_id|>"
                  f"{output}<|eot_id|><|end_of_text|>"
              )
          else:
              formatted_text = (
                  f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>"
                  f"{instruction}<|eot_id|>"
                  f"<|start_header_id|>assistant<|end_header_id|>"
                  f"{output}<|eot_id|><|end_of_text|>"
              )
          formatted_results.append(formatted_text)
      return formatted_results

  def prot_dataset_token_length(self, dataframe, tokenizer, title):
      counter = Counter()
      culumn = dataframe

      for i in tqdm(range(len(culumn))):
          num_tokens = len(tokenizer.tokenize(culumn[i]))
          counter[num_tokens] += 1

      # plot token length distribution
      plt.bar(counter.keys(), counter.values(), width=1.0)
      plt.xlabel("token length")
      plt.ylabel("frequency")
      plt.title(title)
      plt.gca().yaxis.set_major_locator(plt.MaxNLocator(integer=True))
      plt.gca().yaxis.set_major_formatter(plt.FormatStrFormatter("%d"))
      plt.show()

  def tokenizing_dataset(self, tokenizer: AutoTokenizer, dataset):
      tokenized_dataset = dataset.map(lambda example: self.alpaca_tokenize_function(example, tokenizer), batched=False)
      return tokenized_dataset

  def alpaca_tokenize_function(self, example: dict, tokenizer: AutoTokenizer) -> dict:
      text = example["text"]
      full_tokens = tokenizer(text, truncation=True, padding="max_length", max_length=self.dataset_max_length, return_tensors="pt")

      input_ids = full_tokens["input_ids"].squeeze()
      attention_mask = full_tokens["attention_mask"].squeeze()
      assistant_start = "<|start_header_id|>assistant<|end_header_id|>"

      if assistant_start in text:
          prompt_end_pos = text.find(assistant_start) + len(assistant_start)
          prompt_text = text[:prompt_end_pos]

          prompt_tokens = tokenizer(prompt_text, truncation=True, padding=False, add_special_tokens=False)
          labels = input_ids.clone()
          prompt_length = len(prompt_tokens["input_ids"])
          labels[:prompt_length] = -100

      result = {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}
      return result

  def load_alpaca_dataset(self):
      with open(self.alpaca_dataset, 'r', encoding='utf-8') as f:
          data = json.load(f)
          data_count = len(data)
          iloc_data = data[0:int(data_count * self.apply_dataset_rate)]
          dataframe = pandas.DataFrame(iloc_data)
      dataframe = dataframe[['instruction', 'input', 'output']]
      return dataframe

  def split_alpaca_dataset(self, data):
      total_size_dataset = len(data)
      train_size = int(total_size_dataset * self.dataset_train_rate)
      validation_size = int(total_size_dataset * self.dataset_validation_rate)
      test_size = total_size_dataset - (train_size + validation_size)

      train_dataset = data[0:train_size]
      validation_dataset = data[train_size:train_size + validation_size]
      test_dataset = data[train_size + validation_size:total_size_dataset]

      return train_dataset, validation_dataset, test_dataset

  def data_collator(self, tokenizer: AutoTokenizer):
      data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, pad_to_multiple_of=8)
      return data_collator
  
  def lora_config(self):
      lora_config = LoraConfig(
          r=self.lora_r,
          lora_alpha=self.lora_alpha,
          target_modules=self.lora_target_modules,
          lora_dropout=self.lora_dropout,
          bias="none",
          task_type="CAUSAL_LM"
      )
      return lora_config

In [8]:
class DeepSpeedUtil:

 def __init__(self, environment_valiables):
  self.deepspeed_zero0 = environment_valiables["deepspeed_zero0"]
  self.deepspeed_train_config_zero0_path = environment_valiables["deepspeed_train_config_zero0_path"]
  self.deepspeed_zero1 = environment_valiables["deepspeed_zero1"]
  self.deepspeed_train_config_zero1_path = environment_valiables["deepspeed_train_config_zero1_path"]
  self.deepspeed_zero2 = environment_valiables["deepspeed_zero2"]
  self.deepspeed_train_config_zero2_path = environment_valiables["deepspeed_train_config_zero2_path"]
  self.deepspeed_zero3 = environment_valiables["deepspeed_zero3"]
  self.deepspeed_train_config_zero3_path = environment_valiables["deepspeed_train_config_zero3_path"]
  self.deepspeed_zero3_infinity = environment_valiables["deepspeed_zero3_infinity"]
  self.deepspeed_train_config_zero3_infinity_path = environment_valiables["deepspeed_train_config_zero3_infinity_path"]

 def deepspeed_zero_config(self):
  config = (self.deepspeed_train_config_zero0_path if self.deepspeed_zero0 == True else
            self.deepspeed_train_config_zero1_path if self.deepspeed_zero1 == True else
            self.deepspeed_train_config_zero2_path if self.deepspeed_zero2 == True else
            self.deepspeed_train_config_zero3_path if self.deepspeed_zero3 == True else
            self.deepspeed_train_config_zero3_infinity_path if self.deepspeed_zero3_infinity == True else None)
  return config

 def deepspeed_valiable_name(self):
  deepspeed_valiable_name = ("Deepspeed_Zero0" if self.deepspeed_zero0 == True else
                             "Deepspeed_Zero1" if self.deepspeed_zero1 == True else
                             "Deepspeed_Zero2" if self.deepspeed_zero2 == True else
                             "Deepspeed_Zero3" if self.deepspeed_zero3 == True else
                             "Deepspeed_Zero3_Infinity" if self.deepspeed_zero3_infinity == True else None)
  return deepspeed_valiable_name

In [9]:
import time
from datetime import datetime
import socket
import random
import string
import os
import shutil
import wandb
import psutil
import GPUtil
from pynvml import nvmlDeviceGetHandleByIndex, nvmlDeviceGetPowerUsage, nvmlDeviceGetTemperature
import threading
import csv

class LoggingUtil:
    
    def __init__(self, environment_valiables):
        self.stop_event = threading.Event()
        self.environment_valiables = environment_valiables

    def date_time(self):
        result = time.strftime("%Y%m%d%H%M", time.localtime())
        return result
    
    def timestamp(self):
        result = datetime.now().isoformat()
        return result
    
    def get_hostname(self):
        hostname = socket.gethostname()
        return hostname
    
    def split_model_name(self):
        model_name_valiable = self.environment_valiables["model_name"].split('/')[-1]
        return model_name_valiable

    def generate_random_num_strings(self):
        pool_num = "123456789"
        pool_string = string.ascii_letters
        num_strings = pool_num + pool_string
        result = "".join(random.choices(num_strings, k=10))
        return result
    
    def common_directory_path(self, split_model_name, deepspeed_valiable_name, wandb_run_id, date_time):
        common_directory_path_valiable = f"{split_model_name}_{deepspeed_valiable_name}_{wandb_run_id}_{date_time}"
        join_output_directory_path = os.path.join('./output', common_directory_path_valiable)
        os.makedirs(join_output_directory_path, exist_ok=True)
        return join_output_directory_path

    def output_directory_path(self, common_directory_path):
        wandb_output_directory_path = os.path.join(common_directory_path, 'wandb_output')
        wandb_run_name = common_directory_path
        deepspeed_train_output = os.path.join(common_directory_path, 'deepspeed_train_output')
        check_point_output = os.path.join(common_directory_path, 'check_point_output')
        export_train_output = os.path.join(common_directory_path, 'export_train_output')
        save_model_output = os.path.join(common_directory_path, 'save_model_output')
        os.makedirs(export_train_output, exist_ok=True)
        return [wandb_output_directory_path, wandb_run_name, deepspeed_train_output, check_point_output, export_train_output, save_model_output]

    def export_files(self, export_train_output):
        shutil.copy(self.environment_valiables["code_file"], export_train_output)
        export_train_loggings = os.path.join(export_train_output, 'train_loggings.csv')
        export_device_metric_loggings = os.path.join(export_train_output, 'device_metric_loggings.csv')
        export_deepspeed_metric_loggings = os.path.join(export_train_output, 'deepspeed_metric_loggings.csv')
        export_validation_loggings = os.path.join(export_train_output, 'validation_loggings.csv')
        return [export_train_loggings, export_device_metric_loggings, export_deepspeed_metric_loggings, export_validation_loggings]
    
    def init_wandb(self, wandb_output_directory_path, wandb_run_id, wandb_run_name):
        wandb_run = wandb.init(entity=self.environment_valiables["wandb_account_name"], project=self.environment_valiables["wandb_project_name"], dir=wandb_output_directory_path, id=wandb_run_id, name=wandb_run_name, mode='offline')
        return wandb_run
    
    def end_wandb(self):
        wandb.finish()

    def logging_device_cpu(self):
        per_core_usage = psutil.cpu_percent(percpu=True)
        per_core_freqs = psutil.cpu_freq(percpu=True)
        temps = psutil.sensors_temperatures().get('coretemp', [])
        temp_val = temps[0].current if temps else None

        per_core_current_freqs = [f.current for f in per_core_freqs]
        per_core_max_freqs = [f.max for f in per_core_freqs]

        per_core_freq_percentages = [
            (current / max_freq) * 100 if max_freq else None
            for current, max_freq in zip(per_core_current_freqs, per_core_max_freqs)
        ]

        data = {
            'TIMESTAMP': self.timestamp(),
            'CPU_AVG_USAGE_PERCENT': sum(per_core_usage) / len(per_core_usage),
            'CPU_AVG_CLOCK_MHZ': sum(per_core_current_freqs) / len(per_core_current_freqs),
            'CPU_AVG_CLOCK_PERCENT': sum(filter(None, per_core_freq_percentages)) / len(per_core_freq_percentages),
            'CPU_TEMP_C': temp_val
        }

        for i, (u, f, fp) in enumerate(zip(per_core_usage, per_core_current_freqs, per_core_freq_percentages), start=1):
            data[f'CPU_CORE_{i}_USAGE_PERCENT'] = u
            data[f'CPU_CORE_{i}_CLOCK_MHZ'] = f
            data[f'CPU_CORE_{i}_CLOCK_PERCENT'] = fp
        return data
    
    def logging_device_gpu(self):
        gpus = GPUtil.getGPUs()
        gpu_data = {
            'TIMESTAMP': self.timestamp(),
        }

        for i, gpu in enumerate(gpus):
            gpu_handle = nvmlDeviceGetHandleByIndex(i)
            gpu_data[f'GPU_{i}_USAGE_PERCENT'] = gpu.load * 100
            gpu_data[f'GPU_{i}_MEM_USED_MB'] = gpu.memoryUsed
            gpu_data[f'GPU_{i}_MEM_UTIL_PERCENT'] = gpu.memoryUtil * 100
            gpu_data[f'GPU_{i}_POWER_W'] = nvmlDeviceGetPowerUsage(gpu_handle) / 1000.0
            gpu_data[f'GPU_{i}_TEMP_C'] = nvmlDeviceGetTemperature(gpu_handle, 0)
        return gpu_data
    
    def logging_device_ram(self):
        mem = psutil.virtual_memory()
        return {
            'TIMESTAMP': self.timestamp(),
            'RAM_USAGE_PERCENT': mem.percent,
        }

    def logging_device_disk(self):
        usage = psutil.disk_usage('/')
        return {
            'TIMESTAMP': self.timestamp(),
            'DISK_USED_GB': usage.used / (1024**3),
            'DISK_USAGE_PERCENT': usage.percent,
        }
    
    def logging_device_network(self):
        stats = psutil.net_io_counters(pernic=True)
        net = {'TIMESTAMP': self.timestamp()}

        for iface, val in stats.items():
            sent_mb = val.bytes_sent / (1024**2)
            recv_mb = val.bytes_recv / (1024**2)
            net[f'NETWORK_{iface}_SENT_MB'] = sent_mb
            net[f'NETWORK_{iface}_RECV_MB'] = recv_mb
        return net
    
    def write_to_csv_log(self, data, output_dir):
        os.makedirs(os.path.dirname(output_dir), exist_ok=True)

        file_exists = os.path.isfile(output_dir)

        with open(output_dir, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=list(data.keys()))
            if not file_exists:
                writer.writeheader()
            writer.writerow(data)

    def loop_logging_metric_process(self, log_file, wandb_run_name, wandb_run_id):
        while not self.stop_event.is_set():
            cpu = self.logging_device_cpu()
            gpu = self.logging_device_gpu()
            ram = self.logging_device_ram()
            disk = self.logging_device_disk()
            net = self.logging_device_network()

            merged = {
                'wandb_account_name': self.environment_valiables["wandb_account_name"],
                'wandb_project_name': self.environment_valiables["wandb_project_name"],
                'wandb_run_name': wandb_run_name,
                'wandb_run_id': wandb_run_id,
                'hostname': self.get_hostname(),
                    **cpu,
                    **gpu,
                    **ram,
                    **disk,
                    **net,
                }
            self.write_to_csv_log(data=merged, output_dir=log_file)
            time.sleep(self.environment_valiables["logging_distance_time"])


    def start_logging_device_metric(self, log_file, wandb_run_name, wandb_run_id):
            self.stop_event.clear()
            global logging_thread
            logging_thread = threading.Thread(
                target=self.loop_logging_metric_process,
                args=(
                    log_file,
                    wandb_run_name,
                    wandb_run_id
                )
            )
            logging_thread.start()
    
    def finish_logging_device_metric(self):
        self.stop_event.set()
        logging_thread.join()

In [10]:
# Train Metrics Callback
from typing import Any, Dict, Optional
from transformers import TrainerCallback, TrainerState, TrainingArguments, TrainerControl

class TrainMetricsCallback(TrainerCallback):

    def __init__(self, wandb_account_name: str, wandb_project_name: str, wandb_run_name: str, wandb_run_id: str, hostname: str, log_file: str):
        self.wandb_account_name = wandb_account_name
        self.wandb_project_name = wandb_project_name
        self.wandb_run_name = wandb_run_name
        self.wandb_run_id = wandb_run_id
        self.hostname = hostname
        self.log_file = log_file

    def on_log(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, logs: Optional[Dict[str, Any]] = None, **kwargs: Any) -> None:
        logging_util = LoggingUtil(environment_valiables=default_environment_variables)

        train_metrics = {key: logs[key] for key in ['loss', 'grad_norm', 'learning_rate', 'epoch'] if logs and key in logs}

        if all(key in train_metrics for key in ['loss', 'grad_norm', 'learning_rate', 'epoch']):
            metrics = {
                'TIMESTAMP': logging_util.timestamp(),
                'wandb_account_name': self.wandb_account_name,
                'wandb_project_name': self.wandb_project_name,
                'wandb_run_name': self.wandb_run_name,
                'wandb_run_id': self.wandb_run_id,
                'hostname': self.hostname,
                'step': state.global_step,
                **train_metrics
            }
            logging_util.write_to_csv_log(data=metrics, output_dir=self.log_file)

In [11]:
# Validation Metrics Callback
class ValidationMetricsCallback(TrainerCallback):

    def __init__(self, wandb_account_name: str, wandb_project_name: str, wandb_run_name: str, wandb_run_id: str, hostname: str, log_file: str):
        self.wandb_account_name = wandb_account_name
        self.wandb_project_name = wandb_project_name
        self.wandb_run_name = wandb_run_name
        self.wandb_run_id = wandb_run_id
        self.hostname = hostname
        self.log_file = log_file

    def on_log(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, logs: Optional[Dict[str, Any]] = None, **kwargs: Any) -> None:
        if logs and any(key.startswith('eval_') for key in logs.keys()):
            logging_util = LoggingUtil(environment_valiables=default_environment_variables)
                
            validation_metrics = {key: logs[key] for key in ['eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch'] if logs and key in logs}

            if all(key in validation_metrics for key in ['eval_loss', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch']):
                metrics = {
                    'TIMESTAMP': logging_util.timestamp(),
                    'wandb_account_name': self.wandb_account_name,
                    'wandb_project_name': self.wandb_project_name,
                    'wandb_run_name': self.wandb_run_name,
                    'wandb_run_id': self.wandb_run_id,
                    'hostname': self.hostname,
                    'step': state.global_step,
                    **validation_metrics
                }
                logging_util.write_to_csv_log(data=metrics, output_dir=self.log_file)

In [12]:
# DeepSpeed Metrics Callback
class DeepSpeedMetricsCallback(TrainerCallback):

    def __init__(self, trainer, wandb_account_name: str, wandb_project_name: str, wandb_run_name: str, wandb_run_id: str, hostname: str, log_file: str):
        self.trainer = trainer
        self.wandb_account_name = wandb_account_name
        self.wandb_project_name = wandb_project_name
        self.wandb_run_name = wandb_run_name
        self.wandb_run_id = wandb_run_id
        self.hostname = hostname
        self.log_file = log_file
        self.logging_util = LoggingUtil(environment_valiables=default_environment_variables)

    def on_step_end(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs: Any) -> None:
            engine = getattr(self.trainer, "deepspeed", None)
            if engine is None:
                return
            
            timestamp = self.logging_util.timestamp()
        
            if state.global_step % 1000 == 0:
                for model_param_name, model_param in engine.module.named_parameters():
                    param_metrics = {
                        'TIMESTAMP': timestamp,
                        'wandb_account_name': self.wandb_account_name,
                        'wandb_project_name': self.wandb_project_name,
                        'wandb_run_name': self.wandb_run_name,
                        'wandb_run_id': self.wandb_run_id,
                        'hostname': self.hostname,
                        'step': state.global_step,
                        'model_param_name': model_param_name,
                        'model_param_numel': model_param.numel(),
                        'model_param_dtype': model_param.dtype,
                        'model_param_device': model_param.device.type,
                    }
                    self.logging_util.write_to_csv_log(data=param_metrics, output_dir=self.log_file)

In [13]:
import warnings
import torch.distributed as dist
import subprocess
import glob

In [14]:
# Warning Suppression
warnings.filterwarnings("ignore", message="torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly.") # Warning Suppression
warnings.filterwarnings("ignore", message="MatMul8bitLt: inputs will be cast*") # Warning Suppression

In [15]:
# Call Instance
train_util = TrainUtil(default_environment_variables)
logging_util = LoggingUtil(default_environment_variables)
deepspeed_util = DeepSpeedUtil(default_environment_variables)

In [16]:
# Set Variables
date_time = logging_util.date_time()
split_model_name = logging_util.split_model_name()
hostname = logging_util.get_hostname()
timestamp = logging_util.timestamp()
wandb_run_id = logging_util.generate_random_num_strings()
common_directory_path = logging_util.common_directory_path(split_model_name=split_model_name, deepspeed_valiable_name=deepspeed_util.deepspeed_valiable_name(), wandb_run_id=wandb_run_id, date_time=date_time)
wandb_output_directory_path, wandb_run_name, deepspeed_train_output, check_point_output, export_train_output, save_model_output = logging_util.output_directory_path(common_directory_path)
export_train_loggings, export_device_metric_loggings, export_deepspeed_metric_loggings, export_validation_loggings = logging_util.export_files(export_train_output)

In [ ]:
# Login WanDB
wandb_run_name = "/".join(wandb_run_name.split("/")[2:])
wandb_run = logging_util.init_wandb(wandb_output_directory_path=wandb_output_directory_path, wandb_run_id=wandb_run_id, wandb_run_name=wandb_run_name)

In [18]:
# Load Model
tokenizer = AutoTokenizer.from_pretrained(default_environment_variables["model_name"])
tokenizer.add_special_tokens({'pad_token': '<|end_of_text|>'})
model = LlamaForCausalLM.from_pretrained(default_environment_variables["model_name"])
model.config.use_cache = False

In [ ]:
# print model architecture
print(model)

In [ ]:
# apply lora config
from peft import get_peft_model, prepare_model_for_kbit_training

lora_config = train_util.lora_config()
lora_model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
model = get_peft_model(lora_model, lora_config)
print(model.print_trainable_parameters())

In [ ]:
# read json dataset
dataframe = train_util.load_alpaca_dataset()
dataframe.head()

In [ ]:
# replace \n, \r, continuous spaces to single space and remove leading and trailing spaces
cleaned_dataframe = train_util.clean_alpaca_dataset(dataframe=dataframe)
cleaned_dataframe.head()

In [ ]:
normalized_dataframe = train_util.normalize_input_column_alpaca_dataset(dataframe=cleaned_dataframe)
normalized_dataframe.head()

In [ ]:
# check tokens
dataframe_instruction_tokens = train_util.prot_dataset_token_length(dataframe=normalized_dataframe["instruction"], tokenizer=tokenizer, title="instruction_length_token_map")

In [ ]:
# check tokens
dataframe_instruction_tokens = train_util.prot_dataset_token_length(dataframe=normalized_dataframe["input"], tokenizer=tokenizer, title="input_token_length_map")

In [ ]:
# check tokens
dataframe_instruction_tokens = train_util.prot_dataset_token_length(dataframe=normalized_dataframe["output"], tokenizer=tokenizer, title="output_token_length_map")


In [ ]:
# format to qa task
formatted_text = train_util.format_alpaca_dataset(dataframe=normalized_dataframe)
for i in range(len(formatted_text)):
    print(f"formatted_text[{i}]: {formatted_text[i]}")

In [ ]:
# check tokens
dataset_row_tokens = train_util.prot_dataset_token_length(dataframe=formatted_text, tokenizer=tokenizer, title="dataset_token_length_map")

In [ ]:
# DeepSpeed Zero Config
print(deepspeed_util.deepspeed_zero_config())

deepspeed_dataframe = pandas.read_json(deepspeed_util.deepspeed_zero_config(), encoding="UTF-8")
deepspeed_dataframe.head()

In [30]:
# Set Training Arguments
training_args = TrainingArguments(
    output_dir=check_point_output,
    num_train_epochs=default_environment_variables["num_train_epochs"],
    per_device_train_batch_size=default_environment_variables["per_device_train_batch_size"],
    per_device_eval_batch_size=default_environment_variables["per_device_eval_batch_size"],
    gradient_accumulation_steps=default_environment_variables["gradient_accumulation_steps"],
    optim=default_environment_variables["optim"],
    logging_steps=default_environment_variables["logging_steps"],
    learning_rate=default_environment_variables["learning_rate"],
    lr_scheduler_type=default_environment_variables["lr_scheduler_type"],
    warmup_ratio=default_environment_variables["warmup_ratio"],
    seed=default_environment_variables["seed"],
    fp16=default_environment_variables["fp16"],
    bf16=default_environment_variables["bf16"],
    deepspeed=deepspeed_util.deepspeed_zero_config(),
    save_strategy=default_environment_variables["save_strategy"],
    save_steps=default_environment_variables["save_steps"],
    save_total_limit=default_environment_variables["save_total_limit"],
    eval_strategy=default_environment_variables["evaluation_strategy"],
    eval_steps=default_environment_variables["eval_steps"],
    do_eval=default_environment_variables["do_eval"],
)

In [31]:
# # check default model output

# import torch
# from transformers import pipeline

# model_generate_start_time = time.time()

# model_id = default_environment_variables["model_name"]
# pipe = pipeline("text-generation", model=model_id, torch_dtype=torch.bfloat16, device_map="auto", pad_token_id=tokenizer.eos_token_id)
# pipe("what is AI ?")

# model_generate_end_time = time.time()
# model_generate_elapsed_time = model_generate_end_time - model_generate_start_time
# print(f"Model generate elapsed time: {model_generate_elapsed_time:.2f} seconds")

In [32]:
# output normalized_alpaca_data.json

# normalized_dataframe.to_json('normalized_alpaca_data.json', orient='records', force_ascii=False, indent=2)
# json_string = normalized_dataframe.to_json(orient='records', force_ascii=False, indent=2)
# print(json_string)

In [ ]:
# split dataset
train_dataset, validation_dataset, test_dataset = train_util.split_alpaca_dataset(formatted_text)
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(validation_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

In [ ]:
# check dataset
print(train_dataset[0])
print(validation_dataset[0])
print(test_dataset[0])

In [ ]:
# check dataset type
print(type(train_dataset[0]))
print(type(validation_dataset[0]))
print(type(test_dataset[0]))

In [ ]:
# check dataset type
print(type(train_dataset))
print(type(validation_dataset))
print(type(test_dataset))

In [37]:
# convert dataset
from datasets import Dataset
train_hf_dataset = Dataset.from_dict({"text": train_dataset})
validation_hf_dataset = Dataset.from_dict({"text": validation_dataset})
test_hf_dataset = Dataset.from_dict({"text": test_dataset})

In [ ]:
# check dataset
print(f"train_hf_dataset: {train_hf_dataset[0]}")
print(f"validation_hf_dataset: {validation_hf_dataset[0]}")
print(f"test_hf_dataset: {test_hf_dataset[0]}")

In [ ]:
# tokenize
tokenized_train_dataset = train_util.tokenizing_dataset(tokenizer=tokenizer, dataset=train_hf_dataset)
tokenized_validation_dataset = train_util.tokenizing_dataset(tokenizer=tokenizer, dataset=validation_hf_dataset)
tokenized_test_dataset = train_util.tokenizing_dataset(tokenizer=tokenizer, dataset=test_hf_dataset)

In [ ]:
# check tokenize field
print(f"tokenized_train_dataset: {tokenized_train_dataset}")
print(f"tokenized_validation_dataset: {tokenized_validation_dataset}")
print(f"tokenized_test_dataset: {tokenized_test_dataset}")

In [ ]:
# check tokenize text column
print(f"tokenized_train_dataset: {tokenized_train_dataset[0]['text']}")
print(f"tokenized_validation_dataset: {tokenized_validation_dataset[0]['text']}")
print(f"tokenized_test_dataset: {tokenized_test_dataset[0]['text']}")

In [ ]:
# check tokenize input_ids column
print(f"tokenized_train_dataset: {tokenized_train_dataset[0]['input_ids']}")
print(f"tokenized_validation_dataset: {tokenized_validation_dataset[0]['input_ids']}")
print(f"tokenized_test_dataset: {tokenized_test_dataset[0]['input_ids']}")

In [ ]:
# check tokenize attention_mask column
print(f"tokenized_train_dataset: {tokenized_train_dataset[0]['attention_mask']}")
print(f"tokenized_validation_dataset: {tokenized_validation_dataset[0]['attention_mask']}")
print(f"tokenized_test_dataset: {tokenized_test_dataset[0]['attention_mask']}")

In [ ]:
# check tokenize labels column
print(f"tokenized_train_dataset: {tokenized_train_dataset[0]['labels']}")
print(f"tokenized_validation_dataset: {tokenized_validation_dataset[0]['labels']}")
print(f"tokenized_test_dataset: {tokenized_test_dataset[0]['labels']}")

In [45]:
# setting data collator
data_collator = train_util.data_collator(tokenizer=tokenizer)

In [ ]:
# Set Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [47]:
# Set Callback
train_metrics_callback = TrainMetricsCallback(log_file=export_train_loggings, wandb_run_id=wandb_run_id, wandb_project_name=default_environment_variables["wandb_project_name"], wandb_run_name=wandb_run_name, wandb_account_name=default_environment_variables["wandb_account_name"], hostname=hostname)
deepspeed_metrics_callback = DeepSpeedMetricsCallback(trainer=trainer, log_file=export_deepspeed_metric_loggings, wandb_run_id=wandb_run_id, wandb_project_name=default_environment_variables["wandb_project_name"], wandb_run_name=wandb_run_name, wandb_account_name=default_environment_variables["wandb_account_name"], hostname=hostname)
validation_metrics_callback = ValidationMetricsCallback(log_file=export_validation_loggings, wandb_run_id=wandb_run_id, wandb_project_name=default_environment_variables["wandb_project_name"], wandb_run_name=wandb_run_name, wandb_account_name=default_environment_variables["wandb_account_name"], hostname=hostname)
trainer.add_callback(train_metrics_callback)
trainer.add_callback(deepspeed_metrics_callback)
trainer.add_callback(validation_metrics_callback)

In [48]:
# Start logging device metric
logging_util.start_logging_device_metric(log_file=export_device_metric_loggings, wandb_run_name=wandb_run_name, wandb_run_id=wandb_run_id)

In [ ]:
# model train
trainer.train()

In [ ]:
# save model
trainer.save_model(save_model_output)
# save tokenizer
trainer.save_tokenizer(save_model_output)
# save metrics
trainer.save_state(save_model_output)

In [ ]:
# end wandb
logging_util.end_wandb()
# finish logging device metric
logging_util.finish_logging_device_metric()

In [ ]:
# Sync wandb run
wandb_offline_run_dir = os.path.dirname(wandb_run.dir)
subprocess.run(["wandb", "sync", wandb_offline_run_dir])

In [ ]:
program_end_time = time.time()
print(f"program_end_time: {program_end_time - program_start_time}")

In [ ]:
# upload lora model to huggingface
merged_model = model.merge_and_unload()

token = default_environment_variables["huggingface_token"]
repo_name = default_environment_variables["huggingface_repo_name"]
account = default_environment_variables["huggingface_account_name"]

merged_model.push_to_hub(
    repo_name=f"{account}/{repo_name}",
    use_auth_token=token,
    private=True,
    exist_ok=True,
    commit_message="Upload instruction‑tuned model"
)
tokenizer.push_to_hub(
    repo_name=f"{account}/{repo_name}",
    use_auth_token=token,
    private=True,
    exist_ok=True,
    commit_message="Upload instruction‑tuned tokenizer"
)

In [ ]:
import ipykernel
from jupyter_client import KernelManager
import os

# exit kernel
def stop_kernel():
    try:
        # get kernel id
        connection_file = ipykernel.get_connection_file()
        kernel_id = os.path.basename(connection_file).replace('kernel-', '').replace('.json', '')
        
        # stop kernel
        km = KernelManager(kernel_name='python3')
        km.kernel_spec = kernel_id
        km.shutdown_kernel()
        print("Kernel stopped successfully.")
    except Exception as e:
        print(f"Kernel stop error: {e}")

print("Program started")
# stop kernel
stop_kernel()
print("Program ended")